In [70]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math

%matplotlib inline


data_file = "/kaggle/input/chess/games.csv"

data = pd.read_csv(data_file)
data.head()

,id,rated,created_at,last_move_at,turns,victory_status,winner,increment_code,white_id,white_rating,black_id,black_rating,moves,opening_eco,opening_name,opening_ply
0,TZJHLljE,False,1.504210e+12,1.504210e+12,13,outoftime,white,15+2,bourgris,1500,a-00,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,D10,Slav Defense: Exchange Variation,5
1,l1NXvwaE,True,1.504130e+12,1.504130e+12,16,resign,black,5+10,a-00,1322,skinnerua,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,B00,Nimzowitsch Defense: Kennedy Variation,4
2,mIICvQHh,True,1.504130e+12,1.504130e+12,61,mate,white,5+10,ischia,1496,a-00,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,C20,King's Pawn Game: Leonardis Variation,3
3,kWKvrqYL,True,1.504110e+12,1.504110e+12,61,mate,white,20+0,daniamurashov,1439,adivanov2009,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,D02,Queen's Pawn Game: Zukertort Variation,3
4,9tXo1AUZ,True,1.504030e+12,1.504030e+12,95,mate,white,30+3,nik221107,1523,adivanov2009,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,C41,Philidor Defense,5


In [71]:
data = data.drop(['rated', 'created_at', 'last_move_at', 'opening_ply', 'opening_name', 'white_id', 'black_id', 'increment_code', 'opening_eco'], axis=1)
data.head()

,id,turns,victory_status,winner,white_rating,black_rating,moves
0,TZJHLljE,13,outoftime,white,1500,1191,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...
1,l1NXvwaE,16,resign,black,1322,1261,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...
2,mIICvQHh,61,mate,white,1496,1500,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...
3,kWKvrqYL,61,mate,white,1439,1454,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...
4,9tXo1AUZ,95,mate,white,1523,1469,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...


In [72]:
data.describe()

,turns,white_rating,black_rating
count,20058.000000,20058.000000,20058.000000
mean,60.465999,1596.631868,1588.831987
std,33.570585,291.253376,291.036126
min,1.000000,784.000000,789.000000
25%,37.000000,1398.000000,1391.000000
50%,55.000000,1567.000000,1562.000000
75%,79.000000,1793.000000,1784.000000
max,349.000000,2700.000000,2723.000000


In [73]:
print("Number of games with less than 10 moves: {}".format(len(data[data['turns'] <= 10])))
data = data[data['turns'] > 10]

# Create a new column for average rating of the two users
data['rating'] = data.apply(lambda row: (row['white_rating'] + row['black_rating']) / 2, axis=1)
data = data.drop(['white_rating', 'black_rating'], axis=1)
data.head()

Number of games with less than 10 moves: 681


,id,turns,victory_status,winner,moves,rating
0,TZJHLljE,13,outoftime,white,d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...,1345.5
1,l1NXvwaE,16,resign,black,d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...,1291.5
2,mIICvQHh,61,mate,white,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,1498.0
3,kWKvrqYL,61,mate,white,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,1446.5
4,9tXo1AUZ,95,mate,white,e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...,1496.0


Find the most used opening move

I am going to split the dataset into three groups based on the rating (bad, good, expert) to make different level of bots

In [74]:
max_rating = data['rating'].max()
min_rating = data['rating'].min()
delta = int((max_rating - min_rating) / 3)

split_ratings = [min_rating + 1 + (i + 1) * delta for i in range(3)]

split_ratings


[1377.0, 1926.0, 2475.0]

In [75]:
print(data.iloc[0]['moves'])

d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5 Bf4


In [82]:
p = {
    "NONE": 0,
    "R1B": 1, # First black rook
    "N1B": 2, # First black knight
    "B1B": 3, # First black boshop
    "QB": 4, # black queen
    "KB": 5, # black king
    "B2B": 6, # Second black bishop
    "N2B": 7, # Second black knight
    "R2B": 8, # Second black rook
    "P1B": 9, # First black pawn
    "P2B": 10, # First black pawn
    "P3B": 11, # 3 black pawn
    "P4B": 12, # 4 black pawn
    "P5B": 13, # 5 black pawn
    "P6B": 14, # 6 black pawn
    "P7B": 15, # 7 black pawn
    "P8B": 16, # 8 black pawn
    
    "R1W": 17, # First black rook
    "N1W": 18, # First black knight
    "B1W": 19, # First black boshop
    "QW": 20, # black queen
    "KW": 21, # black king
    "B2W": 22, # Second black bishop
    "N2W": 23, # Second black knight
    "R2W": 24, # Second black rook
    "P1W": 25, # First black pawn
    "P2W": 26, # First black pawn
    "P3W": 27, # 3 black pawn
    "P4W": 28, # 4 black pawn
    "P5W": 29, # 5 black pawn
    "P6W": 30, # 6 black pawn
    "P7W": 31, # 7 black pawn
    "P8W": 32 # 8 black pawn
}

state = np.array([
    [ p['R1B'], p['N1B'], p['B1B'], p['QB'],  p['KB'],  p['B2B'], p['N2B'], p['R2B'] ],
    [ p['P1B'], p['P2B'], p['P3B'], p['P4B'], p['P5B'], p['P6B'], p['P7B'], p['P8B'] ],
    [ 0,        0,        0,        0,        0,        0,        0,        0        ],
    [ 0,        0,        0,        0,        0,        0,        0,        0        ],
    [ 0,        0,        0,        0,        0,        0,        0,        0        ],
    [ 0,        0,        0,        0,        0,        0,        0,        0        ],
    [ p['P1W'], p['P2W'], p['P3W'], p['P4W'], p['P5W'], p['P6W'], p['P7W'], p['P8W'] ],
    [ p['R1W'], p['N1W'], p['B1W'], p['QW'],  p['KW'],  p['B2W'], p['N2W'], p['R2W'] ]
])

state

array([[ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [25, 26, 27, 28, 29, 30, 31, 32],
       [17, 18, 19, 20, 21, 22, 23, 24]])

I will need a simple way to switch from chess position (ex: A1, F4) 
to index position (ex: A1 = (7, 0),  F4 = (4, 5))

In [93]:
def chess_pos_to_array_pos(chess_pos):
    letter = chess_pos[0]
    number = int(chess_pos[1])
    row = 8 - number
    col = (ord(letter) - 65)
    
    assert row >= 0 and row <= 7
    assert col >= 0 and col <= 7
    
    return (row, col)

# Testing:
print(chess_pos_to_array_pos("A1"))
print(chess_pos_to_array_pos("A8"))
print(chess_pos_to_array_pos("H1"))
print(chess_pos_to_array_pos("H8"))


(7, 0)
(0, 0)
(7, 7)
(0, 7)


I will also create a method to easily move the piece that is at a certain position to another position

In [ ]:
def move(src, dst):
    src_pos = chess_pos_to_array_pos(src)
    dst_pos = chess_pos_to_array_pos(dst)